In [57]:
# set env variable to json credential file of google cloud

import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/karthikmahendra/Desktop/Salty/salty-trolls-cce24104ff9f.json'

In [58]:
from google.cloud import bigquery
import time
import pandas as pd
import numpy as np

In [59]:
# setup client
bq_client = bigquery.Client() 

In [60]:
#Query for pulling data of a username
QUERY = '''SELECT  * 
        FROM `bigquery-public-data.hacker_news.full_201510`
        WHERE `bigquery-public-data.hacker_news.full_201510`.by = "danmaz74"
        '''

In [61]:
df = bq_client.query(QUERY).to_dataframe()

In [62]:
df.shape

(1680, 13)

In [63]:
df.head()

,by,score,time,title,type,url,text,parent,deleted,dead,descendants,id,ranking
0,danmaz74,NaN,1368003409,None,comment,None,"""HackingItalia"" has much better chances of suc...",5672999.0,None,None,NaN,5673213,None
1,danmaz74,NaN,1362914038,None,comment,None,The (not so) funny thing is that this has been...,5350893.0,None,None,NaN,5351456,None
2,danmaz74,NaN,1380696100,None,comment,None,Absolutely. There is also no clue that the pag...,6479777.0,None,None,NaN,6481042,None
3,danmaz74,NaN,1370794029,None,comment,None,Does anybody have ideas about how a small fore...,5849087.0,None,None,NaN,5849372,None
4,danmaz74,NaN,1383740372,None,comment,None,"Great, I&#x27;ll try your service instead of d...",6681977.0,None,None,NaN,6682294,None


In [17]:
# import html,re

In [44]:
# def decoder(text):
#     """decode HTML text."""
#     unescaped = html.unescape(text)
#     return unescaped


# def kill_html(text):
#     """Remove html flags."""
#     cleanr = re.compile('<.*?>')
#     cleantext = re.sub(cleanr, ' ', text)
#     return cleantext


# def kill_url(text):
#     """Remove URL with regex"""
#     return ''.join(re.sub(r"http\S+", "", text))





In [51]:
# #df['cleaned_comment'] = np.zeros(df.shape[0])
# kill_url(kill_html(decoder()

In [66]:
# to get rid of the weird characters
import html
df['text'] = df['text'].apply(str)

In [67]:
df['text'] = df['text'].apply(lambda x: html.unescape(x))

In [107]:
import re
def remove_html_tags(text):
   """Remove html tags from a string"""
   clean = re.compile('<.*?>')
   return re.sub(clean, '', text)

In [108]:
df['text'] = df['text'].apply(lambda x: remove_html_tags(x))

In [109]:
df.head()

,by,score,time,title,type,url,text,parent,deleted,dead,descendants,id,ranking
0,danmaz74,NaN,1368003409,None,comment,None,"""HackingItalia"" has much better chances of suc...",5672999.0,None,None,NaN,5673213,None
1,danmaz74,NaN,1362914038,None,comment,None,The (not so) funny thing is that this has been...,5350893.0,None,None,NaN,5351456,None
2,danmaz74,NaN,1380696100,None,comment,None,Absolutely. There is also no clue that the pag...,6479777.0,None,None,NaN,6481042,None
3,danmaz74,NaN,1370794029,None,comment,None,Does anybody have ideas about how a small fore...,5849087.0,None,None,NaN,5849372,None
4,danmaz74,NaN,1383740372,None,comment,None,"Great, I'll try your service instead of deskto...",6681977.0,None,None,NaN,6682294,None


In [110]:
# job_config = bigquery.QueryJobConfig()
# now=time.time()
# query_job=bq_client.query(QUERY,location='US')
# res=query_job.result()
# print('query took:',round(time.time()-now,2),'s')

In [111]:
# now=time.time()
# destination_uri = "gs://karthikkm028-ds/*hacker.csv"
# dataset_ref = bq_client.dataset("hacker_news", project='bigquery-public-data')
# table_ref = dataset_ref.table("full_201510")

# extract_job = bq_client.extract_table(
#     table_ref,
#     destination_uri)
# extract_job.result()  # Waits for job to complete
# print('create table and write to GCS took:',round(time.time()-now,2),'s')

In [112]:
# gcs = gcsfs.GCSFileSystem(project='bigquery-public-data') 
# files=gcs.glob(destination_uri)
# df = pd.concat([pd.read_csv('gs://'+f) for f in files], ignore_index=True)
# print('read csv took:',round(time.time()-now,2),'s')

In [113]:

df_drop = df.drop(columns= ['score','title','url','deleted','dead','descendants','ranking'])

In [114]:
df_drop.head()

,by,time,type,text,parent,id
0,danmaz74,1368003409,comment,"""HackingItalia"" has much better chances of suc...",5672999.0,5673213
1,danmaz74,1362914038,comment,The (not so) funny thing is that this has been...,5350893.0,5351456
2,danmaz74,1380696100,comment,Absolutely. There is also no clue that the pag...,6479777.0,6481042
3,danmaz74,1370794029,comment,Does anybody have ideas about how a small fore...,5849087.0,5849372
4,danmaz74,1383740372,comment,"Great, I'll try your service instead of deskto...",6681977.0,6682294


In [115]:
df_drop.shape

(1680, 6)

In [116]:
df_drop.dtypes

by         object
time        int64
type       object
text       object
parent    float64
id          int64
dtype: object

In [117]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [118]:
type(df_drop['text'][0])

str

In [119]:
s = sia.polarity_scores(df_drop['text'][0])

In [120]:
s

{'neg': 0.124, 'neu': 0.636, 'pos': 0.239, 'compound': 0.706}

In [121]:
s['neg']

0.124

In [122]:
sia.polarity_scores(df_drop['text'][0])

{'neg': 0.124, 'neu': 0.636, 'pos': 0.239, 'compound': 0.706}

In [123]:
df_drop['compound']=np.zeros(df_drop.shape[0])
# df_drop['pos']=np.zeros(df_drop.shape[0])
# df_drop['neu']=np.zeros(df_drop.shape[0])

In [124]:
# populate vader sentiments in additional cols
for idx,x in df_drop['text'].iteritems():
    #print (idx,x)
    df_drop['compound'][idx] = sia.polarity_scores(x)['compound']
#     df_drop['pos'][idx] = sia.polarity_scores(x)['pos']
#     df_drop['neu'][idx]= sia.polarity_scores(x)['neu']


/Users/karthikmahendra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [125]:
df_drop.head()

,by,time,type,text,parent,id,compound
0,danmaz74,1368003409,comment,"""HackingItalia"" has much better chances of suc...",5672999.0,5673213,0.7060
1,danmaz74,1362914038,comment,The (not so) funny thing is that this has been...,5350893.0,5351456,0.8727
2,danmaz74,1380696100,comment,Absolutely. There is also no clue that the pag...,6479777.0,6481042,-0.2960
3,danmaz74,1370794029,comment,Does anybody have ideas about how a small fore...,5849087.0,5849372,0.4588
4,danmaz74,1383740372,comment,"Great, I'll try your service instead of deskto...",6681977.0,6682294,0.8555


In [126]:
# # Set threshold of .5 for filtering positive and negative comments. drop comments with value 1 

# df_neg = df_drop[(df_drop['neg']> 0.5) & (df_drop['neg'] !=1)]
# df_pos = df_drop[(df_drop['pos']> 0.5) & (df_drop['pos'] !=1)]

In [127]:
df_drop = df_drop.sort_values(['compound'],ascending=False)
#df_pos =df_pos.sort_values(['pos'],ascending=False)

In [128]:
from datetime import datetime

#convert unix time to datetime object with accuracy of secs

df_drop['time']=pd.to_datetime(df_drop['time'],unit='s')

In [129]:
# top 10 positive comments
df_pos = df_drop.head(10)

In [130]:
# # top 10 negative comments
df_neg = df_drop.tail(10)

In [131]:
df_pos = df_pos.sort_values(['time'],ascending=False)
df_neg = df_neg.sort_values(['time'],ascending=False)

In [132]:
df_pos

,by,time,type,text,parent,id,compound
1654,danmaz74,2014-12-19 08:22:15,comment,"There isn't an easy way imho, because this was...",8771114.0,8771471,0.9923
343,danmaz74,2014-11-04 12:08:35,comment,"Yes, but advertising plays a big role on paid ...",8556020.0,8556127,0.9809
1070,danmaz74,2013-10-14 17:46:41,comment,"I don't agree with the article in general, but...",6548469.0,6548583,0.9890
38,danmaz74,2013-03-21 07:05:10,comment,"The wise little worker ant doesn't have money,...",5408876.0,5413448,0.9954
46,danmaz74,2012-06-04 18:24:39,comment,I didn't have the time to study the project in...,4064462.0,4065162,0.9850
1346,danmaz74,2012-03-14 10:17:50,comment,But it is Google itself that is creating the i...,3702045.0,3702504,0.9913
363,danmaz74,2011-12-26 20:54:11,comment,"As this is in beta, if it sells or not is some...",3393644.0,3393909,0.9916
498,danmaz74,2011-12-26 18:23:06,comment,"Thank you for your suggestions.The ""Executive ...",3393433.0,3393485,0.9867
1186,danmaz74,2011-11-23 17:23:14,comment,"In Italy salaries for tech jobs are higher, bu...",3270096.0,3270778,0.9923
506,danmaz74,2011-10-13 11:57:22,comment,"I'm not talking about a small village, but a s...",3106744.0,3106890,0.9826


In [133]:
df_neg

,by,time,type,text,parent,id,compound
883,danmaz74,2015-04-14 08:05:56,comment,Oh God what a perfect conspiracy! But it's not...,9370481.0,9372761,-0.9792
831,danmaz74,2015-02-14 11:37:40,comment,> If automated cars drove perfectly 99.9999% o...,9049054.0,9049413,-0.9894
1585,danmaz74,2015-01-15 07:21:19,comment,"As an Italian, I have to say that Italy isn't ...",8890952.0,8891305,-0.9271
170,danmaz74,2014-07-24 08:21:50,comment,"I live in Europe, I think that employees need ...",8078570.0,8078650,-0.9352
424,danmaz74,2014-03-01 18:21:57,comment,"First of all, the Ukrainian government wasn't ...",7325325.0,7325404,-0.9247
962,danmaz74,2013-09-08 19:28:05,comment,"Yes, but to know ""why"" (I'd much rather say ""h...",6349815.0,6349925,-0.9402
868,danmaz74,2013-07-19 15:07:53,comment,The thing is that there will always be people ...,6070396.0,6070625,-0.9709
1371,danmaz74,2013-05-13 16:16:44,comment,What's more despicable about killing children ...,5698915.0,5699533,-0.9484
1287,danmaz74,2012-07-13 11:45:49,comment,"If you say don't blame only the hackers, I agr...",4239281.0,4239344,-0.9353
1548,danmaz74,2011-11-26 19:34:51,comment,There is so much shortsightedness in this whol...,3280344.0,3280490,-0.9340
